<a href="https://colab.research.google.com/github/markkod/pc3-enzymes/blob/exploratory_da/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#preinstalled version of pytorch has to be the same as the pre-compiled versions of the pytorch-geometric packages that we download later on.
#versions might change quickly, so if you get a strange error later on, check the torch version of Google colab later on as follows:

import torch
torch.__version__

'1.7.0+cu101'

In [2]:
# Script to generate variations of the kernels yourself
# https://ucloud.univie.ac.at/index.php/s/E3YKph0jkpbw8TN


# #Download the TUDataset Repository with
!git clone https://github.com/chrsmrrs/tudataset.git
# #move this script to tudataset/tud_benchmark

# #Install pytorch geometric: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
# #Here is the gpu cuda installation, for the cpu version replace cu102 with cpu
%pip --no-cache-dir install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-geometric

%pip --no-cache-dir install pybind11
!sudo apt-get install libeigen3-dev



fatal: destination path 'tudataset' already exists and is not an empty directory.
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 40.1MB/s 
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 1.3MB/s 
  Found existing installation: torch-sparse 0.6.8
    Uninstalling torch-sparse-0.6.8:
      Successfully uninstalled torch-sparse-0.6.8
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 1.2MB/s 
  Found existing installation: torch-cluster 1.5.8
    Uninstalling torch-cluster-1.5.8:
      Successfully uninstalled torch-cluster-1.5.8
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 48.8MB/s 


In [3]:
%cd ..
%cd /content/tudataset/tud_benchmark/kernel_baselines/
! ls
! g++ -I /usr/include/eigen3 -O3 -shared -std=c++11 -fPIC `python3 -m pybind11 --includes`  kernel_baselines.cpp src/*cpp -o ../kernel_baselines`python3-config --extension-suffix`
%cd ..

/
/content/tudataset/tud_benchmark/kernel_baselines
kernel_baselines.cpp  src
/content/tudataset/tud_benchmark


In [4]:
!ls -al /usr/local/cuda

lrwxrwxrwx 1 root root 9 Dec 21 17:24 /usr/local/cuda -> cuda-10.1


In [5]:
import os
import torch
import numpy as np
import kernel_baselines as kb
import auxiliarymethods
from auxiliarymethods import datasets as dp
from scipy.sparse import save_npz


In [6]:
def setup_directory(dir_name, verbose=False):
    """
    Setup directory in case it does not exist
    Parameters:
    -------------
    dir_name: str, path + name to directory
    verbose: bool, indicates whether directory creation should be printed or not.
    """
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name)
            if verbose:
                print("Created Directory: {}".format(dir_name))
        except Exception as e:
            raise RuntimeError(
                "Could not create directory: {}\n {}".format(dir_name, e))


In [79]:
use_edge_labels = False
for USE_LABELS in [True, False]:# Except IMDB-BINARY
    for dataset, use_labels in [["ENZYMES", USE_LABELS]]:
        if use_labels:
            base_path = os.path.join("kernels","node_labels")
        else:
            base_path = os.path.join("kernels","without_labels")
        setup_directory(base_path)
        print("Start processing data set ", dataset)
        # Download dataset.
        classes = dp.get_dataset(dataset)
        # *Weisfeihler-Lehman*
        print("Start computing Weisfeihler-Lehman gram matrix and vector representations")
        iterations = 6
        #0 taking just the nodelabels themselves into account; 
        #1 considers nearest-neighbours, 2 one layer deeper and so on
        for i in range(1, iterations):
            print("Start iteration ", i)
            #Gram Matrix for the Weisfeiler-Lehman subtree kernel
            gram_matrix_wl = kb.compute_wl_1_dense(dataset, i, use_labels, use_edge_labels)
            np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_wl{i}.csv"),
                    gram_matrix_wl,
                    delimiter=";")
            #Sparse Vectors for the Weisfeiler-Lehmann subtree kernel
            vectors_wl = kb.compute_wl_1_sparse(dataset, i, use_labels, use_edge_labels)
            save_npz(os.path.join(base_path,f"{dataset}_vectors_wl{i}.npz"),
                    vectors_wl, compressed=True)

        # *Graphlet kernel*
        print("Start computing Graphlet gram matrix")

        #Gram Matrix for the Graphlet kernel
        gram_matrix_graphlet= kb.compute_graphlet_dense(dataset, use_labels, use_edge_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_graphlet.csv"),
                gram_matrix_graphlet,
                delimiter=";")

        print("Start computing Graphlet vector representation")
        #Sparse Vectors for the Graphlet kernel
        vectors_graphlet = kb.compute_graphlet_sparse(dataset, use_labels, use_edge_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_graphlet.npz"),
                vectors_graphlet, compressed=True)


        print("Start computing Shortest path gram matrix")

        #Gram Matrix for the Shortest path kernel
        gram_matrix_shortestpath = kb.compute_shortestpath_dense(dataset, use_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_shortestpath.csv"),
                gram_matrix_shortestpath,
                delimiter=";")

        print("Start computing Shortest path vector representation")

        #Sparse Vectors for the Shortest path kernel
        vectors_shortestpath = kb.compute_shortestpath_sparse(dataset, use_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_shortestpath.npz"),
                vectors_shortestpath, compressed=True)



Start processing data set  ENZYMES
Start computing Weisfeihler-Lehman gram matrix and vector representations
Start iteration  1
Start iteration  2
Start iteration  3
Start iteration  4
Start iteration  5
Start computing Graphlet gram matrix
Start computing Graphlet vector representation
Start computing Shortest path gram matrix
[[ 73924.  30546.  35514. ...  79166. 117398.  96672.]
 [ 30546.  15744.  16882. ...  36894.  56388.  46384.]
 [ 35514.  16882.  21316. ...  33136.  54940.  44536.]
 ...
 [ 79166.  36894.  33136. ... 316796. 227164. 199264.]
 [117398.  56388.  54940. ... 227164. 337252. 289668.]
 [ 96672.  46384.  44536. ... 199264. 289668. 256200.]]
Start computing Shortest path vector representation
Start processing data set  ENZYMES
Start computing Weisfeihler-Lehman gram matrix and vector representations
Start iteration  1
Start iteration  2
Start iteration  3
Start iteration  4
Start iteration  5
Start computing Graphlet gram matrix
Start computing Graphlet vector represent

In [8]:
import pandas as pd

df1 = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_wl1.csv', sep=';', header=None)
df2 = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_wl2.csv', sep=';', header=None)
df3 = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_wl3.csv', sep=';', header=None)
df4 = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_wl4.csv', sep=';', header=None)
df5 = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_wl5.csv', sep=';', header=None)



In [9]:
from auxiliarymethods import auxiliary_methods as aux
from auxiliarymethods import kernel_evaluation as ke

dfs = [df1, df2, df3, df3, df4, df5]
classes = dp.get_dataset('ENZYMES')
all_matrices = []
for df in dfs:
  df_np = df.to_numpy()
  gm = aux.normalize_gram_matrix(df_np)
  all_matrices.append(gm)

accuracy, std_10, std_100 = ke.kernel_svm_evaluation(all_matrices, classes, num_repetitions=10, all_std=True)

Streaming output truncated to the last 5000 lines.
val_acc 51.85185185185185
c 10
val_acc 50.0
c 1
val_acc 25.925925925925924
c 0.1
val_acc 16.666666666666664
c 0.01
val_acc 16.666666666666664
c 0.001
val_acc 16.666666666666664
c 1000
val_acc 50.0
c 100
val_acc 50.0
c 10
val_acc 50.0
c 1
val_acc 25.925925925925924
c 0.1
val_acc 16.666666666666664
c 0.01
val_acc 16.666666666666664
c 0.001
val_acc 16.666666666666664
c 1000
val_acc 48.148148148148145
c 100
val_acc 48.148148148148145
c 10
val_acc 46.2962962962963
c 1
val_acc 27.77777777777778
c 0.1
val_acc 16.666666666666664
c 0.01
val_acc 16.666666666666664
c 0.001
val_acc 16.666666666666664
c 1000
val_acc 29.629629629629626
c 100
val_acc 33.33333333333333
c 10
val_acc 33.33333333333333
c 1
val_acc 25.925925925925924
c 0.1
val_acc 12.962962962962962
c 0.01
val_acc 14.814814814814813
c 0.001
val_acc 14.814814814814813
c 1000
val_acc 44.44444444444444
c 100
val_acc 44.44444444444444
c 10
val_acc 38.88888888888889
c 1
val_acc 27.777777777777

In [10]:
sp = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_shortestpath.csv', sep=';', header=None).to_numpy()
graphlet = pd.read_csv('/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_gram_matrix_graphlet.csv', sep=';', header=None).to_numpy()

baseline_kernels = {
    'WL': dfs,
    'SP': [sp],
    'GRAPHLET': [graphlet]
}


In [16]:
sp_normalized = [aux.normalize_gram_matrix(sp)]
accuracy_sp, std_10_sp, std_100_sp = ke.kernel_svm_evaluation(sp_normalized, classes, num_repetitions=10, all_std=True)


0
c 1000
val_acc 42.592592592592595
c 100
val_acc 50.0
c 10
val_acc 44.44444444444444
c 1
val_acc 42.592592592592595
c 0.1
val_acc 20.37037037037037
c 0.01
val_acc 14.814814814814813
c 0.001
val_acc 14.814814814814813
c 1000
val_acc 35.18518518518518
c 100
val_acc 37.03703703703704
c 10
val_acc 31.48148148148148
c 1
val_acc 29.629629629629626
c 0.1
val_acc 24.074074074074073
c 0.01
val_acc 9.25925925925926
c 0.001
val_acc 9.25925925925926
c 1000
val_acc 42.592592592592595
c 100
val_acc 51.85185185185185
c 10
val_acc 38.88888888888889
c 1
val_acc 42.592592592592595
c 0.1
val_acc 14.814814814814813
c 0.01
val_acc 9.25925925925926
c 0.001
val_acc 9.25925925925926
c 1000
val_acc 48.148148148148145
c 100
val_acc 38.88888888888889
c 10
val_acc 37.03703703703704
c 1
val_acc 37.03703703703704
c 0.1
val_acc 14.814814814814813
c 0.01
val_acc 16.666666666666664
c 0.001
val_acc 16.666666666666664
c 1000
val_acc 31.48148148148148
c 100
val_acc 37.03703703703704
c 10
val_acc 37.03703703703704
c 1
va

In [64]:
def find_keys_with_condition(data, cond):
    return list(filter(lambda x: cond in x, data.keys()))

def load_data():
  result = {}
  extensions = ['csv', 'npz']
  types = ['gram_matrix', 'vectors']
  algos = ['wl1', 'wl2', 'wl3', 'wl4', 'wl5', 'shortestpath', 'graphlet']
  base_name = '/content/tudataset/tud_benchmark/kernels/node_labels/ENZYMES_{0}_{1}.{2}'
  for t, e in zip(types, extensions):
    for a in algos:
      file_name = base_name.format(t, a, e)
      if e == 'csv':
        f = np.loadtxt(file_name, delimiter=';')
      else:
        f = np.load(file_name)
      key = a + '_' + t
      result[key] = f
  final_result = {
      'gram_matrix': {
          'wl': [],
          'shortestpath': [],
          'graphlet': []
      },
      'vectors': {
          'wl': [],
          'shortestpath': [],
          'graphlet': []
          }
  }
  for key in result:
    file_type = 'vectors' if 'vectors' in key else 'gram_matrix'
    substr_idx = key.find('_')
    new_key = key[:substr_idx]
    if 'wl' in key:
      final_result[file_type]['wl'].append(result[key])
    else:
      final_result[file_type][new_key].append(result[key])
  return final_result




In [65]:
data = load_data()

In [56]:
data

{'gram_matrix': {'graphlet': [array([[11177.,  6596.,  4333., ..., 11081., 13908., 10164.],
          [ 6596.,  4879.,  3005., ...,  4882.,  5786.,  3278.],
          [ 4333.,  3005.,  2132., ...,  2814.,  3236.,  1674.],
          ...,
          [11081.,  4882.,  2814., ..., 20397., 22060., 18291.],
          [13908.,  5786.,  3236., ..., 22060., 28850., 24883.],
          [10164.,  3278.,  1674., ..., 18291., 24883., 22359.]])],
  'shortestpath': [array([[ 73924.,  30546.,  35514., ...,  79166., 117398.,  96672.],
          [ 30546.,  15744.,  16882., ...,  36894.,  56388.,  46384.],
          [ 35514.,  16882.,  21316., ...,  33136.,  54940.,  44536.],
          ...,
          [ 79166.,  36894.,  33136., ..., 316796., 227164., 199264.],
          [117398.,  56388.,  54940., ..., 227164., 337252., 289668.],
          [ 96672.,  46384.,  44536., ..., 199264., 289668., 256200.]])],
  'wl': [array([[ 874.,  502.,  585., ...,  973.,  998.,  935.],
          [ 502.,  360.,  362., ...,  59

In [15]:
 feature_vectors_graphlet = data['graphlet_vectors']
 accuracy, std_10, std_100 = lin

SyntaxError: ignored

In [46]:
def eval_kernel(kernel, classes, mode, n_reps=10, all_std=True):
  normalized = []
  for array in kernel:
    if mode == 'LINEAR':
      normalized.append(aux.normalize_feature_vector(array))
    else:
      normalized.append(aux.normalize_gram_matrix(array))
  if mode == 'LINEAR':
    return ke.linear_svm_evaluation(normalized, classes, num_repetitions=n_reps, all_std=all_std)
  return ke.kernel_svm_evaluation(normalized, classes, num_repetitions=n_reps, all_std=all_std)

def eval_all(data):
  classes = dp.get_dataset('ENZYMES')
  result = {
      'gram_matrix': {},
      'vectors': {}
      }
  for key in data.keys():
    mode = 'LINEAR' if key == 'vectors' else 'KERNEL'
    print('Calculating', mode)
    for a in data[key]:
      print(a)
      result[key][a] = eval_kernel(data[key][a], classes, mode)
  return result



In [57]:
eval_all(data)

TypeError: ignored

In [45]:
data

{'gram_matrix': {'graphlet': [array([[11177.,  6596.,  4333., ..., 11081., 13908., 10164.],
          [ 6596.,  4879.,  3005., ...,  4882.,  5786.,  3278.],
          [ 4333.,  3005.,  2132., ...,  2814.,  3236.,  1674.],
          ...,
          [11081.,  4882.,  2814., ..., 20397., 22060., 18291.],
          [13908.,  5786.,  3236., ..., 22060., 28850., 24883.],
          [10164.,  3278.,  1674., ..., 18291., 24883., 22359.]])],
  'shortestpath': [array([[ 73924.,  30546.,  35514., ...,  79166., 117398.,  96672.],
          [ 30546.,  15744.,  16882., ...,  36894.,  56388.,  46384.],
          [ 35514.,  16882.,  21316., ...,  33136.,  54940.,  44536.],
          ...,
          [ 79166.,  36894.,  33136., ..., 316796., 227164., 199264.],
          [117398.,  56388.,  54940., ..., 227164., 337252., 289668.],
          [ 96672.,  46384.,  44536., ..., 199264., 289668., 256200.]])],
  'wl': [array([[ 874.,  502.,  585., ...,  973.,  998.,  935.],
          [ 502.,  360.,  362., ...,  59

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
list(data['vectors']['graphlet'][0].keys())

['indices', 'indptr', 'format', 'shape', 'data']

In [75]:
data['vectors']['graphlet'][0]['shape']

array([ 600, 3455])

In [78]:
data['vectors']['graphlet'][0]['data'].reshape((600, 3455))

ValueError: ignored